In [6]:
import fit

ModuleNotFoundError: No module named 'prfpy_cfdn'

In [5]:
import sys
# append current python modules' folder path
# example: need to import module.py present in '/path/to/python/module/not/in/syspath'
sys.path.append('/tank/klundert/projects/cfdn/prfpy_cfdn/')